<a href="https://colab.research.google.com/github/shivanikikkuru-beep/shivani_reddy/blob/main/exp_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import random
from sympy import isprime, nextprime, mod_inverse, factorint

class DSAAlg:
    @staticmethod
    def get_next_prime(seed: int) -> int:
        """Return the next prime >= seed using sympy.nextprime for reliability."""
        return nextprime(seed - 1)

    @staticmethod
    def find_q(n: int) -> int:
        """
        Find a prime q that divides n (i.e. a prime factor of n).
        For DSA we need q | (p-1). We return the smallest prime factor > 1.
        """
        if n <= 2:
            raise ValueError("n must be > 2 to find a prime factor")
        # factorint returns a dict {prime: exponent}
        factors = factorint(n)
        primes = sorted(factors.keys())
        for prime in primes:
            if isprime(prime):
                return int(prime)
        raise ValueError("No prime factor found (unexpected)")

    @staticmethod
    def get_gen(p: int, q: int) -> int:
        """
        Choose g = h^{(p-1)/q} mod p with h in [2, p-2]. Ensure g > 1.
        Repeat until valid generator found.
        """
        if q <= 1 or p <= 2:
            raise ValueError("Invalid p or q")
        while True:
            h = random.randint(2, p - 2)
            g = pow(h, (p - 1) // q, p)
            if g > 1:
                return g

    @staticmethod
    def main():
        # choose an approximate seed for p; in practice p is very large
        p = DSAAlg.get_next_prime(10600)   # small for demo; use large primes for real use
        # find a q that divides p-1 (a prime factor)
        q = DSAAlg.find_q(p - 1)
        g = DSAAlg.get_gen(p, q)

        print("\nSimulation of Digital Signature Algorithm (DSA) — demo only\n")
        print("Global public key components:")
        print(f" p = {p}")
        print(f" q = {q}")
        print(f" g = {g}\n")

        # Private key x in [1, q-1]
        x = random.randint(1, q - 1)
        # Public key y = g^x mod p
        y = pow(g, x, p)

        # Choose per-message secret k in [1, q-1] (must be random and unique for each signature)
        k = random.randint(1, q - 1)
        # r = (g^k mod p) mod q
        r = pow(g, k, p) % q

        # For demo we use a random integer as the "hash" value H(m)
        # In real DSA use a proper hash (SHA-256 or similar), then reduce mod q as needed.
        hash_val = random.randint(1, q - 1)

        try:
            k_inv = mod_inverse(k, q)
        except Exception as exc:
            print("Failed to compute inverse of k mod q; choose a new k. Error:", exc)
            return

        s = (k_inv * (hash_val + x * r)) % q

        print("Secret / public values:")
        print(f" x (private) = {x}")
        print(f" k (per-message secret) = {k}")
        print(f" y (public) = {y}")
        print(f" H(m) (pseudo-hash) = {hash_val}\n")

        print("Generated signature (r, s):")
        print(f" r = {r}")
        print(f" s = {s}\n")

        # Verification
        try:
            w = mod_inverse(s, q)
        except Exception as exc:
            print("Failed to compute inverse of s mod q; signature invalid. Error:", exc)
            return

        u1 = (hash_val * w) % q
        u2 = (r * w) % q
        v = (pow(g, u1, p) * pow(y, u2, p) % p) % q

        print("Verification checkpoints:")
        print(f" w  = {w}")
        print(f" u1 = {u1}")
        print(f" u2 = {u2}")
        print(f" v  = {v}\n")

        if v == r:
            print(f"Success: signature verified (v == r == {r}).")
        else:
            print("Failure: signature not verified (v != r).")

if __name__ == "__main__":
    DSAAlg.main()



Simulation of Digital Signature Algorithm (DSA) — demo only

Global public key components:
 p = 10601
 q = 2
 g = 10600

Secret / public values:
 x (private) = 1
 k (per-message secret) = 1
 y (public) = 10600
 H(m) (pseudo-hash) = 1

Generated signature (r, s):
 r = 0
 s = 1

Verification checkpoints:
 w  = 1
 u1 = 1
 u2 = 0
 v  = 0

Success: signature verified (v == r == 0).
